In [1]:
import os
import time
import json
import math
import requests
import logging
import pyotp
import yaml
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from api_helper import ShoonyaApiPy

In [2]:
with open('cred.yml') as f:
    cred = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
    token = cred['token']
    user = cred['user']
    pwd = cred['pwd']
    factor2 = pyotp.TOTP(token).now()
    vc = cred['vc']
    app_key = cred['apikey']
    imei = cred['imei']

    nse_exchange = "NSE"
    nse_token = "26000"
    bse_exchange = "BSE"
    bse_token = "1"
    vix_token = "26017"
    days = 1
    interval = "1"

In [4]:
    api = ShoonyaApiPy()
    api.login(userid=user, password=pwd, twoFA=factor2, vendor_code=vc, api_secret=app_key, imei=imei)
    print("Login Successful!")

Login Successful!


In [5]:


    def get_time_series(exchange, token, days, interval):
        now = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
        prev_day = now - timedelta(days=days)
        ret = api.get_time_price_series(exchange=exchange, token=token, starttime=prev_day.timestamp(), interval=interval)
        return pd.DataFrame(ret) if ret else None
    
    def process_df(df):
        df = df.sort_index(ascending=False)
        df[['intl', 'inth']] = df[['intl', 'inth']].apply(pd.to_numeric)
        return df

In [6]:
    nse_df = get_time_series(nse_exchange, nse_token, days, interval)
    bse_df = get_time_series(bse_exchange, bse_token, days, interval)

    nse_df['time'] = pd.to_datetime(nse_df['time'], format='%d-%m-%Y %H:%M:%S')
    nse_pdc = float(nse_df.sort_values(by='time', ascending=False).iloc[0]['intc'])

    bse_df['time'] = pd.to_datetime(bse_df['time'], format='%d-%m-%Y %H:%M:%S')
    bse_pdc = float(bse_df.sort_values(by='time', ascending=False).iloc[0]['intc'])

TypeError: 'NoneType' object is not subscriptable

In [ ]:
    nse_df = process_df(nse_df)
    bse_df = process_df(bse_df)

    nse_resp = api.get_quotes(exchange=nse_exchange, token=nse_token)
    bse_resp = api.get_quotes(exchange=bse_exchange, token=bse_token)
    vix_resp = api.get_quotes(exchange=nse_exchange, token=vix_token)

    nse_ltp = round(float(nse_resp['lp']),2)
    nse_pdh = round(float(nse_df['inth'].max()),2)
    nse_pdl = round(float(nse_df['intl'].min()),2)
    nse_pdc = round(nse_pdc,2)
    nse_cdh = round(float(nse_resp.get('h', nse_resp['c'])),2)
    nse_cdl = round(float(nse_resp.get('l', nse_resp['c'])),2)
    nse_hh = round(nse_ltp - nse_pdl,2)
    nse_ll = round(nse_pdh - nse_ltp,2)
    nse_ill = round(nse_cdh - nse_ltp,2)
    nse_ihh = round(nse_ltp - nse_cdl,2)
    nse_c2c = nse_ltp - nse_pdc
    

    bse_ltp = round(float(bse_resp['lp']),2)
    bse_pdh = round(float(bse_df['inth'].max()),2)
    bse_pdl = round(float(bse_df['intl'].min()),2)
    bse_pdc = round(bse_pdc,2)
    bse_cdh = round(float(bse_resp.get('h', bse_resp['c'])),2)
    bse_cdl = round(float(bse_resp.get('l', bse_resp['c'])),2)
    bse_hh = round(bse_ltp - bse_pdl,2)
    bse_ll = round(bse_pdh - bse_ltp,2)
    bse_ill = round(bse_cdh - bse_ltp,2)
    bse_ihh = round(bse_ltp - bse_cdl,2)
    bse_c2c = bse_ltp - bse_pdc


    vix = float(vix_resp['lp'])

In [ ]:
print("nse_ltp  : " ,nse_ltp)
print("nse_pdh  : " ,nse_pdh)
print("nse_pdl  : " ,nse_pdl)
print("nse_cdh  : " ,nse_cdh)
print("nse_cdl  : " ,nse_cdl)
print("nse_hh  : " ,nse_hh)
print("nse_ll  : " ,nse_ll)
print("nse_ihh  : " ,nse_ihh)
print("nse_c2c  : " ,nse_ll)
print("vix : ", vix)

nse_ltp  :  24853.15
nse_pdh  :  24908.85
nse_pdl  :  24614.65
nse_cdh  :  24853.15
nse_cdl  :  24853.15
nse_hh  :  238.5
nse_ll  :  55.7
nse_ihh  :  0.0
nse_c2c  :  55.7
vix :  17.28


In [ ]:
print("bse_ltp  : " ,bse_ltp)
print("bse_pdh  : " ,bse_pdh)
print("bse_pdl  : " ,bse_pdl)
print("bse_cdh  : " ,bse_cdh)
print("bse_cdl  : " ,bse_cdl)
print("bse_hh  : " ,bse_hh)
print("bse_ll  : " ,bse_ll)
print("bse_ihh  : " ,bse_ihh)
print("bse_c2c  : " ,bse_ll)

bse_ltp  :  81721.08
bse_pdh  :  81904.19
bse_pdl  :  80904.08
bse_cdh  :  81721.08
bse_cdl  :  81721.08
bse_hh  :  817.0
bse_ll  :  183.11
bse_ihh  :  0.0
bse_c2c  :  183.11


In [ ]:
(exchange, tradingsymbol, strikeprice, count)

api.get_option_chain(exchange=nse_exchange, token=nse_token)